In [8]:
!pip install pandas
!pip install -U plotly kaleido
!pip install nbformat

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 MB 12.0 MB/s eta 0:00:0000:0100:01
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.8/337.8 kB 21.7 MB/s eta 0:00:00


In [1]:
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


CATEGORIES = ["Writing", "Roleplay", "Reasoning", "Math", "Coding", "Extraction", "STEM", "Humanities"]


def get_model_df():
    cnt = 0
    q2result = []
    fin = open("gpt-4_single.jsonl", "r")
    for line in fin:
        obj = json.loads(line)
        obj["category"] = CATEGORIES[(obj["question_id"]-81)//10]
        q2result.append(obj)
    df = pd.DataFrame(q2result)
    return df

df = get_model_df()

In [2]:
df

,question_id,model,judge,user_prompt,judgment,score,turn,tstamp,category
0,141,Llama-2-7b-chat,"[gpt-4, single-v1-multi-turn]",<|The Start of Assistant A's Conversation with...,The assistant's response is thorough and accur...,9.0,2,1.689729e+09,STEM
1,88,Llama-2-7b-chat,"[gpt-4, single-v1]",[Instruction]\nPlease act as an impartial judg...,"The assistant's response is creative, engaging...",9.0,1,1.689729e+09,Writing
2,143,Llama-2-7b-chat,"[gpt-4, single-v1-multi-turn]",<|The Start of Assistant A's Conversation with...,"The assistant's response is highly detailed, a...",10.0,2,1.689729e+09,STEM
3,153,Llama-2-7b-chat,"[gpt-4, single-v1-multi-turn]",<|The Start of Assistant A's Conversation with...,"The assistant's response is detailed, accurate...",9.0,2,1.689729e+09,Humanities
4,132,Llama-2-7b-chat,"[gpt-4, single-v1-multi-turn]",<|The Start of Assistant A's Conversation with...,"The assistant's response is accurate, relevant...",10.0,2,1.689729e+09,Extraction
...,...,...,...,...,...,...,...,...,...
315,118,lyre-lm,"[gpt-4, single-math-v1]",[Instruction]\nPlease act as an impartial judg...,The assistant's answer is incorrect. The assis...,1.0,1,1.701214e+09,Math
316,158,lyre-lm,"[gpt-4, single-v1]",[Instruction]\nPlease act as an impartial judg...,"The assistant's response is comprehensive, acc...",9.0,1,1.701214e+09,Humanities
317,148,lyre-lm,"[gpt-4, single-v1-multi-turn]",<|The Start of Assistant A's Conversation with...,The assistant's response is detailed and compr...,6.0,2,1.701214e+09,STEM
318,121,lyre-lm,"[gpt-4, single-math-v1]",[Instruction]\nPlease act as an impartial judg...,The assistant's answer is incorrect. The assis...,2.0,1,1.701214e+09,Coding


In [3]:
all_models = df["model"].unique()
print(all_models)
scores_all = []
for model in all_models:
    for cat in CATEGORIES:
        # filter category/model, and score format error (<1% case)
        res = df[(df["category"]==cat) & (df["model"]==model) & (df["score"] >= 0)]
        score = res["score"].mean()

        # scores_all.append({"model": model, "category": cat, "score": score, "winrate": winrate, "wtrate": winrate_adjusted})
        scores_all.append({"model": model, "category": cat, "score": score})

['Llama-2-7b-chat' 'lyre-lm']


In [5]:
# target_models = ['llama-7b-finetune-max-small', 'llama-7b-finetune-max', "llama-7b-finetune-all", "llama-7b-finetune-top-01-percent", "llama-7b-finetune-clustered-top-01-percent", 'llama-7b-finetune-clustered-top-10-percent','llama-7b-finetune-top-10-percent','Llama-2-7b-chat', "gpt-4"]
# target_models = ['llama-7b-finetune-max-small', 'llama-7b-finetune-max', "llama-7b-finetune-all", 'llama-7b-finetune-clustered-top-10-percent','llama-7b-finetune-top-10-percent','Llama-2-7b-chat', "gpt-4"]

target_models = ['Llama-2-7b-chat', 'lyre-lm']

scores_target = [scores_all[i] for i in range(len(scores_all)) if scores_all[i]["model"] in target_models]

# sort by target_models
scores_target = sorted(scores_target, key=lambda x: target_models.index(x["model"]), reverse=True)

df_score = pd.DataFrame(scores_target)
df_score = df_score[df_score["model"].isin(target_models)]

# rename_map = {'llama-7b-finetune-max-small': 'llama-7b-finetune-max-small',
#               'llama-7b-finetune-max': 'llama-7b-finetune-max',
#               "gpt-4": "GPT-4",
#               "llama-7b-finetune-clustered-top-01-percent":"llama-7b-finetune-clustered-top-01-percent",
#               "llama-7b-finetune-top-01-percent":"llama-7b-finetune-top-01-percent",
#               'Llama-2-7b-chat':'Llama-2-7b-chat',
#               'llama-7b-finetune-clustered-top-10-percent':'llama-7b-finetune-clustered-top-10-percent',
#               'llama-7b-finetune-top-10-percent':'llama-7b-finetune-top-10-percent'}
rename_map = {'Llama-2-7b-chat': 'Llama-2-7B-chat',
              'lyre-lm': 'Lyre-LM'}

for k, v in rename_map.items():
    df_score.replace(k, v, inplace=True)

fig = px.line_polar(df_score, r = 'score', theta = 'category', line_close = True, category_orders = {"category": CATEGORIES},
                    color = 'model', markers=True, color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_layout(
    autosize=False,
    width=800,
    height=700,
)


fig.show()

In [11]:
df = df[df['turn'] == 1]

In [12]:
all_models = df["model"].unique()
print(all_models)
scores_all = []
for model in all_models:
    for cat in CATEGORIES:
        # filter category/model, and score format error (<1% case)
        res = df[(df["category"]==cat) & (df["model"]==model) & (df["score"] >= 0)]
        score = res["score"].mean()

        # scores_all.append({"model": model, "category": cat, "score": score, "winrate": winrate, "wtrate": winrate_adjusted})
        scores_all.append({"model": model, "category": cat, "score": score})

['Llama-2-7b-chat' 'lyre-lm']


In [13]:
# target_models = ['llama-7b-finetune-max-small', 'llama-7b-finetune-max', "llama-7b-finetune-all", "llama-7b-finetune-top-01-percent", "llama-7b-finetune-clustered-top-01-percent", 'llama-7b-finetune-clustered-top-10-percent','llama-7b-finetune-top-10-percent','Llama-2-7b-chat', "gpt-4"]
# target_models = ['llama-7b-finetune-max-small', 'llama-7b-finetune-max', "llama-7b-finetune-all", 'llama-7b-finetune-clustered-top-10-percent','llama-7b-finetune-top-10-percent','Llama-2-7b-chat', "gpt-4"]

target_models = ['Llama-2-7b-chat', 'lyre-lm']

scores_target = [scores_all[i] for i in range(len(scores_all)) if scores_all[i]["model"] in target_models]

# sort by target_models
scores_target = sorted(scores_target, key=lambda x: target_models.index(x["model"]), reverse=True)

df_score = pd.DataFrame(scores_target)
df_score = df_score[df_score["model"].isin(target_models)]

# rename_map = {'llama-7b-finetune-max-small': 'llama-7b-finetune-max-small',
#               'llama-7b-finetune-max': 'llama-7b-finetune-max',
#               "gpt-4": "GPT-4",
#               "llama-7b-finetune-clustered-top-01-percent":"llama-7b-finetune-clustered-top-01-percent",
#               "llama-7b-finetune-top-01-percent":"llama-7b-finetune-top-01-percent",
#               'Llama-2-7b-chat':'Llama-2-7b-chat',
#               'llama-7b-finetune-clustered-top-10-percent':'llama-7b-finetune-clustered-top-10-percent',
#               'llama-7b-finetune-top-10-percent':'llama-7b-finetune-top-10-percent'}
rename_map = {'Llama-2-7b-chat': 'Llama-2-7B-chat',
              'lyre-lm': 'Lyre-LM'}

for k, v in rename_map.items():
    df_score.replace(k, v, inplace=True)

fig = px.line_polar(df_score, r = 'score', theta = 'category', line_close = True, category_orders = {"category": CATEGORIES},
                    color = 'model', markers=True, color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_layout(
    autosize=False,
    width=800,
    height=700,
)


fig.show()